In [ ]:
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json
from tqdm import tqdm

with open('/content/drive/MyDrive/파이널_3팀/0. 수집데이터/데이터 원본/서울시 휴게음식점 인허가 정보.json') as f:
  d= json.load(f)
df = json_normalize(d, 'DATA')

In [3]:
# 영업 중인 상점 정보만 불러오기
df=df[df['dtlstatenm']=='영업']
df=df.reset_index(drop=True)
df['uptaenm'].unique()

array(['커피숍', '기타 휴게음식점', '편의점', '백화점', '푸드트럭', '패스트푸드', '다방', '일반조리판매',
       '전통찻집', '아이스크림', '떡카페', '철도역구내', '유원지', '키즈카페', '과자점', '고속도로',
       '관광호텔', '극장', '단란주점', '호프/통닭', '공항'], dtype=object)

In [4]:
# 커피숍, 기타 휴게음식점, 키즈카페로 신고된 상점 정보만 불러오기
df=df[(df['uptaenm']=='커피숍')|(df['uptaenm']=='기타 휴게음식점')|(df['uptaenm']=='키즈카페')]
df=df[['bplcnm','rdnwhladdr']]
df['수정주소'] = df['rdnwhladdr'] #위도 경도를 구하기 위해 정제된 도로명주소 칼럼을 별도로 생성
df=df.reset_index(drop=True)
df=df.astype({'수정주소':'str'})

In [5]:
# 결측지가 있는 행과 스타벅스로 신고된 행 제거
df['rdnwhladdr'].replace('', np.nan, inplace=True)
df.dropna(subset=['rdnwhladdr'], inplace=True)
df=df[~(df['bplcnm'].str.contains('스타벅스'))]
df

,bplcnm,rdnwhladdr,수정주소
0,눞커피,"서울특별시 성동구 천호대로 292, 1층 (용답동)","서울특별시 성동구 천호대로 292, 1층 (용답동)"
1,팜스365,"서울특별시 마포구 월드컵로 240, 홈플러스 월드컵점 2층 x9.5,yuw호 (성산동)","서울특별시 마포구 월드컵로 240, 홈플러스 월드컵점 2층 x9.5,yuw호 (성산동)"
2,메가엠지씨커피 종로숭인점,"서울특별시 종로구 종로 377, 1층 (숭인동)","서울특별시 종로구 종로 377, 1층 (숭인동)"
3,노우즈 창덕,"서울특별시 종로구 돈화문로 88-1, UD빌딩 1층 (와룡동)","서울특별시 종로구 돈화문로 88-1, UD빌딩 1층 (와룡동)"
4,대단한 탕후루 성동금호점,"서울특별시 성동구 금호로 148-1, 1층 (금호동1가)","서울특별시 성동구 금호로 148-1, 1층 (금호동1가)"
...,...,...,...
22940,배스킨라빈스 약수역점,"서울특별시 중구 다산로 118 (신당동,(1층))","서울특별시 중구 다산로 118 (신당동,(1층))"
22941,어니스트[:EARNEST],"서울특별시 강서구 공항대로55길 33, 성명빌딩 1층 (등촌동)","서울특별시 강서구 공항대로55길 33, 성명빌딩 1층 (등촌동)"
22942,청담커피 앤 토스트,"서울특별시 강남구 테헤란로 313, 성지하이츠1 지하1층 6호 (역삼동)","서울특별시 강남구 테헤란로 313, 성지하이츠1 지하1층 6호 (역삼동)"
22943,행복상점,"서울특별시 송파구 삼전로8길 5, 1층 (삼전동)","서울특별시 송파구 삼전로8길 5, 1층 (삼전동)"


In [6]:
# 콤마가 포함된 주소가 있을 경우, 콤마 이후의 주소 삭제(1층, 2층 등)
df['수정주소'] = df['수정주소'].map(lambda x : x.split(',')[0] if x.find(',') > 0 else x)

# 괄호가 포함된 주소가 있을 경우, 괄호 이후의 주소 삭제((역삼동) 등)
df['수정주소'] = df['수정주소'].map(lambda x : x.split('(')[0] if x.find('(') > 0 else x)

In [7]:
df

,bplcnm,rdnwhladdr,수정주소
0,눞커피,"서울특별시 성동구 천호대로 292, 1층 (용답동)",서울특별시 성동구 천호대로 292
1,팜스365,"서울특별시 마포구 월드컵로 240, 홈플러스 월드컵점 2층 x9.5,yuw호 (성산동)",서울특별시 마포구 월드컵로 240
2,메가엠지씨커피 종로숭인점,"서울특별시 종로구 종로 377, 1층 (숭인동)",서울특별시 종로구 종로 377
3,노우즈 창덕,"서울특별시 종로구 돈화문로 88-1, UD빌딩 1층 (와룡동)",서울특별시 종로구 돈화문로 88-1
4,대단한 탕후루 성동금호점,"서울특별시 성동구 금호로 148-1, 1층 (금호동1가)",서울특별시 성동구 금호로 148-1
...,...,...,...
22940,배스킨라빈스 약수역점,"서울특별시 중구 다산로 118 (신당동,(1층))",서울특별시 중구 다산로 118
22941,어니스트[:EARNEST],"서울특별시 강서구 공항대로55길 33, 성명빌딩 1층 (등촌동)",서울특별시 강서구 공항대로55길 33
22942,청담커피 앤 토스트,"서울특별시 강남구 테헤란로 313, 성지하이츠1 지하1층 6호 (역삼동)",서울특별시 강남구 테헤란로 313
22943,행복상점,"서울특별시 송파구 삼전로8길 5, 1층 (삼전동)",서울특별시 송파구 삼전로8길 5
